In [1]:
# Importer le package requests
import requests
import pandas as pd
from pymongo import MongoClient
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA

## Extraction de données

Extraire les données depuis l'API de AirQuino

In [2]:
# l'URL de l'API 
url = r'https://airqino-api.magentalab.it/v3/getStationHourlyAvg/283164601'

# Effectuer la requête GET
response = requests.get(url) 

# Vérifier si la réponse est réussi 
if response.status_code == 200:
    # Traiter les données JSON
    data = response.json()
else:
    print(f"Erreur : {response.status_code}")

In [3]:
# convertion des données en dataFrame 
df = pd.DataFrame(data['data'])

# Affichage du tableau 
df.head()

,timestamp,CO,T,T. int.,NO2,O3,PM10,PM2.5,RH
0,2023-09-15 16:00:00,0.0,28.186666,389.36667,19.086350,28.631033,13.600000,3.133333,77.54000
1,2023-09-15 17:00:00,0.0,27.443335,381.36667,19.367783,26.835968,17.266666,4.500000,81.10000
2,2023-09-15 18:00:00,0.0,26.913334,374.90000,19.750890,24.654322,16.400000,4.200000,85.25667
3,2023-09-15 19:00:00,0.0,26.503334,370.36667,20.310297,21.759440,21.900000,8.200000,86.87334
4,2023-09-15 20:00:00,0.0,26.223333,367.63333,19.783640,24.070265,14.866667,5.000000,90.67667


In [5]:
# Convertir le timestamp en format datetime 
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [6]:
# Extraire la date seulement
df['date'] = df['timestamp'].dt.date

In [7]:
# Calculer la moyenne par jour pour chaque capteur 
daily_avg = df.groupby(['date'])[['CO','PM2.5']].mean().reset_index()

daily_avg.head()

,date,CO,PM2.5
0,2023-09-15,0.000000,4.720833
1,2023-09-16,0.000000,3.240771
2,2023-09-17,0.000000,1.936201
3,2023-09-18,0.001158,5.365412
4,2023-09-19,0.000000,6.589651


In [8]:
# Connexion a MongoDB 
client = MongoClient('mongodb://localhost:27017/') 
db = client['data354']
collection = db['daily_averages']

In [9]:
# Préparer les données a insérer dans MongoDB 
records = []
for _,row in daily_avg.iterrows():
    record = {
        "Station_name": data['header']['station_name'], 
        "date": row['date'].strftime('%Y-%m-%d'), 
        "CO_avg": row['CO'], 
        "PM2.5_avg": row['PM2.5'],
        "station_lat": data['header']['station_lat'],
        "station_lon": data['header']['station_lon']
    }
    records.append(record)

# Insérer les données dans la collection MongoDB 
collection.insert_many(records)

print("Données insérées avec succès dans MongoDB!")

Données insérées avec succès dans MongoDB!


In [10]:
# daily_avg.to_csv('challenge354.csv', index=False)

In [11]:
# df.to_csv('data.csv', index=False)

In [12]:
# Convertir la colonne timestamp en datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Définir le timestamp comme index
df.set_index('timestamp', inplace=True)

# Définir explicitement la fréquence à 'h' pour les données horaires
df = df.asfreq('h')

# Appliquer ARIMA pour prédire CO
model_co = ARIMA(df['CO'], order=(5, 1, 0))  
model_co_fit = model_co.fit()

# Appliquer ARIMA pour prédire PM2.5
model_pm25 = ARIMA(df['PM2.5'], order=(5, 1, 0)) 
model_pm25_fit = model_pm25.fit()

# Faire des prévisions pour les 2 prochaines heures pour CO
forecast_co = model_co_fit.forecast(steps=2)
print("Prévisions ARIMA pour CO:", forecast_co, "\n")

# Faire des prévisions pour les 2 prochaines heures pour PM2.5
forecast_pm25 = model_pm25_fit.forecast(steps=2)
print("Prévisions ARIMA pour PM2.5:", forecast_pm25)

Prévisions ARIMA pour CO: 2024-09-14 16:00:00    0.0
2024-09-14 17:00:00    0.0
Freq: h, Name: predicted_mean, dtype: float64
Prévisions ARIMA pour PM2.5: 2024-09-14 16:00:00    2.062890
2024-09-14 17:00:00    2.105639
Freq: h, Name: predicted_mean, dtype: float64
